In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

matplotlib.rc('font', family='Malgun Gothic')  # 윈도우 기준, 맑은 고딕

# 1. 데이터 불러오기
df = pd.read_excel(r'C:\Users\rlaxo\Desktop\2025_Airbnb_NYC_listings.xlsx')

# 2. 가격 전처리: $, , 제거 → float형으로 변환
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)

# 3. 필요한 컬럼만 추출하고 결측치 제거
df = df[['price', 'review_scores_rating']].dropna()

# 4. 로그 변환
df['log_price'] = np.log1p(df['price'])

# 5. IQR 기반 이상치 제거 (log_price 기준)
Q1 = df['log_price'].quantile(0.25)
Q3 = df['log_price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 6. 이상치 제거된 데이터 추출
filtered = df[(df['log_price'] >= lower_bound) & (df['log_price'] <= upper_bound)]

# 7. 로그 → 원래 가격으로 역변환
filtered['price_reversed'] = np.expm1(filtered['log_price'])  # log1p의 역함수

# 가격 구간 나누기 (예: $0~$50, $50~$100, ...)
bins = list(range(0, 1001, 50))  # 0 ~ 1000까지 50단위로 나누기
labels = [f"${b}~${b+50}" for b in bins[:-1]]
filtered['price_range'] = pd.cut(filtered['price_reversed'], bins=bins, labels=labels, right=False)

# 가격대별 숙소 수 집계 (Series)
price_distribution = filtered['price_range'].value_counts().sort_index()

# 결과 출력
print("가격대별 숙소 수 분포:")
print(price_distribution)

# 8. 시각화: 로그 변환 + 이상치 제거 후 분포
plt.figure(figsize=(8, 5))
sns.histplot(filtered['log_price'], bins=50, kde=True, color='purple')
plt.title('로그 변환 + 이상치 제거 후 가격 분포')
plt.xlabel('Log(1 + Price)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()